In [1]:
# This block allows us to import from the benchmark folder,
# as if it was a package installed using pip
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from util import datasets, methods, models
from benchmark.infidelity import infidelity
import itertools
from os import path
import pandas as pd
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

In [3]:
# General parameters
batch_size = 16
n_batches = 16
n_perturbations = 1
data_root = "../data"
use_logits = True
dataset_name = "MNIST"
normalize_attrs = True
aggregation_fn = "avg"

In [4]:
# CIFAR10 parameters
if dataset_name == "CIFAR10":
    dataset = datasets.Cifar(batch_size=batch_size, data_location=path.join(data_root, "CIFAR10"), download=False, shuffle=False, version="cifar10")
    model = models.CifarResnet(version="resnet20", params_loc=path.join(data_root, "models/CIFAR10_resnet20.pth"), num_classes=10, output_logits=use_logits)
    model.eval()

In [5]:
# MNIST parameters
if dataset_name == "MNIST":
    dataset = datasets.MNIST(batch_size=batch_size, data_location=path.join(data_root, "MNIST"), download=False, shuffle=False)
    model = models.MNISTCNN(params_loc=path.join(data_root, "models/MNIST_cnn.pth"), output_logits=use_logits)
    model.eval()

In [6]:
kwargs = {
    "normalize": normalize_attrs,
    "aggregation_fn": aggregation_fn

}

attribution_methods = {
    "GuidedGradCAM": methods.GuidedGradCAM(model, model.get_last_conv_layer(), **kwargs),
    "Gradient": methods.Gradient(model, **kwargs),
    "InputXGradient": methods.InputXGradient(model, **kwargs),
    "IntegratedGradients": methods.IntegratedGradients(model, **kwargs),
    "GuidedBackprop": methods.GuidedBackprop(model, **kwargs),
    "Deconvolution": methods.Deconvolution(model, **kwargs),
}

In [7]:
result = infidelity(data=itertools.islice(dataset.get_dataloader(train=False), n_batches), model=model,
                    methods=attribution_methods, n_perturbations=n_perturbations, pixel_level=normalize_attrs)

Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10
Batch 11
Batch 12
Batch 13
Batch 14
Batch 15
Batch 16


/home/arne/Documents/PhD/projects/benchmark/venv/lib/python3.6/site-packages/captum/attr/_utils/gradient.py:33: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  "required_grads has been set automatically." % index
/home/arne/Documents/PhD/projects/benchmark/venv/lib/python3.6/site-packages/captum/attr/_core/guided_backprop_deconvnet.py:56: UserWarning: Setting backward hooks on ReLU activations.The hooks will be removed after the attribution is finished
  "Setting backward hooks on ReLU activations."


In [8]:
result_df = pd.DataFrame.from_dict(
    {m_name: pd.Series(data=result[m_name]) for m_name in attribution_methods}
)

In [9]:
result_df.mean()

GuidedGradCAM          164.828598
Gradient               169.017807
InputXGradient         166.917404
IntegratedGradients    170.609848
GuidedBackprop         164.475479
Deconvolution          182.588013
dtype: float32